# Basic Low Rank MDPs Oracles

In this notebook we will implement some basic Low-Rank MDPs both finite and infinite such as Simplex Feature MDPs and Block MDPs. This environments will be the "truth" that we want to approximate.

## Simplex MDPs
### Finite State and Action Space
Since in this case we have finite State and Action Spaces, we will use a one hot enconding representation of this spaces, were each element of this sets is assign a number in $\mathcal{N}_0$

In [2]:
import numpy as np
A = 8 #Number of actions
S = 10 #Number of states
d = 3 #Rank of the MDP (Number of Latent Variables in this case)

In [3]:
def proj_mat_to_simplex(M, d=1):
    """This function projects all the column vectors of the Matrix `M` (m x n) onto 
    the standard simplex of dimension `m`. Taken from: 
    https://www.researchgate.net/publication/343831904_NumPy_SciPy_Recipes_for_Data_Science_Projections_onto_the_Standard_Simplex 
    A more simple way would be to just use a softmax over a random vector.
    """
    
    m, n = M.shape

    S = np.sort(M, axis=0)[::-1]
    C = np.cumsum(S, axis=0) - d
    H = S - C / (np.arange(m) + 1).reshape(m, 1)
    H[H <= 0] = np.inf

    r = np.argmin(H, axis=0)
    t = C[r, np.arange(n)] / (r + 1)

    Y = M - t
    Y[Y < 0] = 0
    return Y



Generate Feature Space from random vector

In [16]:
from scipy.special import softmax
# M = np.random.uniform(low=0, high=d, size=(d, S * A)) #One d-dimensional column vector for each (s, a) pair
# P = proj_mat_to_simplex(M).T.reshape((S, A, d))
M = np.random.uniform(low=-d, high=d, size=(S, A, d)) 
P = softmax(M, axis=2)
Phi = lambda s,a: P[s, a, :] #Phi(S_0, A_0) \in R^d
print(Phi(0,0))
print(sum(Phi(0,0)))

[0.15185472 0.60637257 0.24177272]
1.0


Generate a posteriori distribution

In [29]:
U = np.random.uniform(0, 3, (S, d)) #One vector for each dimension 
U = softmax(U, axis=0)
Mu = lambda s:  U[s, :].T # s entre 0 y S-1 
print(Mu(0))
print(sum(U[:,0]))


[0.26021562 0.07329655 0.22941355]
0.9999999999999999


In [106]:
T = lambda s, a, s_: np.dot(Phi(s, a), Mu(s_))
print(T(0, 2, 1)) #Probabilidad de pasar de s_0 a s_1 por medio de s_2

0.7807669790859032


Verify regularity conditions of our construction

In [31]:
integral = np.zeros(d)

for s in range(S):
    integral += Mu(s)

norm_sqr = np.linalg.norm(integral) ** 2
print(norm_sqr)
print(np.isclose(norm_sqr, d, rtol=0.01))

2.9999999999999996
True


In [32]:
import numpy as np
from scipy.stats.distributions import norm

def unif_over_states(states=100):
    return np.random.randint(low=0, high=states)

class SimplexEnvironment:
    def __init__(self, states=100, actions=20, bell_rank=10, intial_distrib=unif_over_states):
        self.S = states
        self.A = actions
        self.d = bell_rank
        self.initial_distrib = intial_distrib

        M = np.random.uniform(0, 3, (self.d, self.S * self.A)) #One d-dimensional column vector for each (s, a) pair
        
        self.P = self.proj_mat_to_simplex(M).T.reshape((self.S, self.A, self.d))
        self.Phi = lambda s,a: self.P[s, a, :]

        U = np.random.uniform(0, 3, (self.S, self.d)) #One vector for each dimension 
        self.U = self.proj_mat_to_simplex(U)
        self.Mu = lambda s:  self.U[s, :].T # s entre 0 y S-1 
        self.T = lambda s, a, s_: np.dot(self.Phi(s, a), self.Mu(s_))



    def proj_mat_to_simplex(self, M, d=1):
        """This function projects all the column vectors of the Matrix `M` (m x n) onto 
        the standard simplex of dimension `m`. Taken from: 
        https://www.researchgate.net/publication/343831904_NumPy_SciPy_Recipes_for_Data_Science_Projections_onto_the_Standard_Simplex """
        m, n = M.shape

        S = np.sort(M, axis=0)[::-1]
        C = np.cumsum(S, axis=0) - d
        H = S - C / (np.arange(m) + 1).reshape(m, 1)
        H[H <= 0] = np.inf

        r = np.argmin(H, axis=0)
        t = C[r, np.arange(n)] / (r + 1)

        Y = M - t
        Y[Y < 0] = 0
        return Y

    def next_step_distrib(self, s, a):
        return np.dot(self.Phi(s,a), self.U.T)

    def next_step_state(self, s, a):
        return np.random.choice(self.S, p=self.next_step_distrib(s,a))
    
    def next_step_reward(self, s, a):
        return norm.rvs()
    
    def next_step(self, s, a):
        """ 
        Gives the next state and reword for the given actions. 
        If `s` is the absorving state (i.e s == self.S - 1. Just by convention)
        Then this will return 0 as reward and a state following the initial state distrib.
        """
        if s == self.S - 1:
            return self.first_step(), 0
        else:
            return self.next_step_state(s, a), self.next_step_reward(s, a)
    
    def first_step(self):
        return self.initial_distrib(self.S)


Generate an Environment from the Simplex MDP

In [33]:
env = SimplexEnvironment()
env.next_step(0,0)
# np.sum(env.P, axis=2)

(90, 1.002155018673309)

In [75]:
env.next_step(0,0)

(3, -1.9631144612288514)

### Simulation

We have created a utility file in where there is a more complete version of the previous simplex environment.

Know we will create some MDPs with such class and some walk data for each one following a uniform policy over actions. That will be the data we use in the implementation of the function approximation. Also is important that we save the MDPs so we can compare later the different algorithms.

Small MDP

In [5]:
from utils import SimplexEnvironment
from pickle import dump
import os

smallMDP = SimplexEnvironment(states=100, actions=10, bell_rank=3)
dump(smallMDP, open(os.path.join("data", "mdps", "small" ,"small_mdp.bin"), 'wb'))

mediumMDP = SimplexEnvironment(states=400, actions=50, bell_rank=5)
dump(mediumMDP, open(os.path.join("data", "mdps", "medium" ,"medium_mdp.bin"), 'wb'))

largeMDP = SimplexEnvironment(states=1000, actions=100, bell_rank=10)
dump(largeMDP, open(os.path.join("data", "mdps", "large" ,"large_mdp.bin"), 'wb'))


In [3]:
%%time
from math import ceil
from pickle import load, dump
import numpy as np
smallMDP = load(open(os.path.join("data", "mdps", "small" ,"small_mdp.bin"), 'rb'))
n = 30*ceil(smallMDP.A * smallMDP.d * smallMDP.S * np.log(smallMDP.S))
path = smallMDP.simulate_n_steps(n)
dump(path, open(os.path.join("data", "mdps", "small" ,"small_mdp_path.bin"), 'wb'))


Simulating 414480 Steps: 100%|██████████| 414480/414480 [00:21<00:00, 19515.94it/s]


CPU times: total: 21.8 s
Wall time: 21.4 s


In [18]:
# Train test splits
from pickle import load, dump
from random import shuffle
from math import ceil
small_path = load( open(os.path.join("data", "mdps", "small" ,"small_mdp_path.bin"), 'rb'))
shuffle(small_path)
train_index = ceil(len(small_path) * 0.7)
print(train_index, len(small_path))

small_train_path = small_path[:train_index]
small_test_path = small_path[train_index:] 

dump(small_train_path, open(os.path.join("data", "mdps", "small" ,"small_mdp_path_train.bin"), 'wb'))
dump(small_test_path, open(os.path.join("data", "mdps", "small" ,"small_mdp_path_test.bin"), 'wb'))


290136 414480


In [4]:
%%time
from math import ceil
from pickle import load, dump
import numpy as np
mediumMDP = load(open(os.path.join("data", "mdps", "medium" ,"medium_mdp.bin"), 'rb'))
n = 30*ceil(mediumMDP.A * mediumMDP.d * mediumMDP.S * np.log(mediumMDP.S))
path = mediumMDP.simulate_n_steps(n)
dump(path, open(os.path.join("data", "mdps", "medium" ,"medium_mdp_path.bin"), 'wb'))

Simulating 17974410 Steps: 100%|██████████| 17974410/17974410 [14:41<00:00, 20396.97it/s]


CPU times: total: 14min 58s
Wall time: 14min 48s


Caculate Optimal Policy a Value Function for the MDPs

In [1]:
%%time
from utils import PolicyIteration
from pickle import load, dump
import numpy as np

MDP = load( open(os.path.join("data", "mdps", "small" ,"small_mdp.bin"), 'rb'))


def reward(s_prev, a, s_next):
    return np.log(s_prev + 1) + np.log(a + 1) + np.log(s_next + 1)

def next_state_prob(s, a):
    return MDP.Phi(s, a) @ MDP.U.T

pol_iter = PolicyIteration(MDP.S , MDP.A, next_state_prob , reward)

v, pol = pol_iter.run()
saving = (v, pol)
dump(saving, open(os.path.join("data", "mdps", "small" ,"small_mdp_optimals.bin"), 'wb'))

  0%|          | 2/2000 [00:00<00:09, 200.06it/s]

CPU times: total: 1.34 s
Wall time: 1.54 s


## Block MDPs